<a href="https://colab.research.google.com/github/Jgallear/CSSP_brazil_23_24/blob/main/forecast_all_simple_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Forecast across brazil but use harvested area threshold to filter out grid cells below 75th percentile of harvested area
!pip install cartopy
!pip install rioxarray

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping
import cartopy.crs as ccrs
from cartopy.feature import ShapelyFeature
from cartopy.io.shapereader import Reader
from sklearn.inspection import PartialDependenceDisplay
from shapely.geometry import Point
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr
from cartopy.feature import ShapelyFeature
from cartopy.io.shapereader import Reader
from rasterio import CRS
from shapely.geometry import mapping
import xarray as xr
from statsmodels.graphics.tsaplots import plot_acf
from keras import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
import functools
import pickle
import cartopy.feature as cf
from scipy.stats import pearsonr

In [ ]:
# load VHI and weather data table
df = pd.read_csv('/content/drive/MyDrive/rs_data/VHI_merged_weather_data.csv')
print(df)

In [ ]:
# load SPEI 1 and 3 data
spei_1df = pd.read_csv('/content/drive/MyDrive/rs_data/SPEI/brazil_spei_025.csv')
spei_2df = pd.read_csv('/content/drive/MyDrive/rs_data/SPEI/brazil_2m_spei_025.csv')
spei_3df = pd.read_csv('/content/drive/MyDrive/rs_data/SPEI/brazil_3m_spei_025.csv')

In [ ]:
spei_1df.drop(columns='time',inplace=True)
spei_3df.drop(columns='time',inplace=True)
spei_2df.drop(columns='time',inplace=True)
spei_1df.rename(columns={'spei':'spei1m'},inplace=True)
spei_2df.rename(columns={'spei':'spei2m'},inplace=True)
spei_3df.rename(columns={'spei':'spei3m'},inplace=True)

spei_dfs = [spei_1df,spei_2df,spei_3df]
spei_df = functools.reduce(lambda left, right: pd.merge(left, right, on=['lat','lon','year','month']), spei_dfs)

#spei_df = spei_1df.merge(spei_3df,on=['lat','lon','year','month'])

In [ ]:
df = df.merge(spei_df,on=['lon','lat','year','month'])

In [ ]:
# Need to add SPI and era5 data to df
e5_longrad = pd.read_csv('/content/drive/MyDrive/rs_data/era5/regridded_era5_longrad.csv')
e5_t2m = pd.read_csv('/content/drive/MyDrive/rs_data/era5/regridded_era5_2m_temperature.csv')
e5_pev = pd.read_csv('/content/drive/MyDrive/rs_data/era5/regridded_era5_pot_evap.csv')

In [ ]:
print(e5_pev)

In [ ]:
e5_longrad.drop(columns=['VHI','Unnamed: 0','precip','rtzsm_inst'],inplace=True)
e5_t2m.drop(columns=['VHI','Unnamed: 0','precip','rtzsm_inst','msdwlwrf','pev'],inplace=True)
e5_pev.drop(columns=['VHI','Unnamed: 0','precip','rtzsm_inst','msdwlwrf'],inplace=True)

In [ ]:
print(e5_longrad)

In [ ]:
print(df)

In [ ]:
# merge to create era5 df
era5_dfs = [e5_longrad,e5_t2m,e5_pev]
e5_df_final = functools.reduce(lambda left, right: pd.merge(left, right, on=['lat','lon','year','month']), era5_dfs)

In [ ]:
df = df.merge(e5_df_final,on=['lon','lat','year','month'])

In [ ]:
print(df)

In [ ]:
# add spi1-3 to the dataframe
spi01 = pd.read_csv('/content/drive/MyDrive/rs_data/spi_gpcc/spi01_regridded_025.csv')
spi02 = pd.read_csv('/content/drive/MyDrive/rs_data/spi_gpcc/spi02_regridded_025.csv')
spi03 = pd.read_csv('/content/drive/MyDrive/rs_data/spi_gpcc/spi03_regridded_025.csv')

In [ ]:
print(spi03)

In [ ]:
# spi03 contains everything so use that
spi03.drop(columns=['VHI','precip','rtzsm_inst'],inplace=True)

In [ ]:
df = df.merge(spi03,on=['lon','lat','year','month'])

In [ ]:
print(df)

In [ ]:
def groupby_gc(df):
  df_list=[]
  for index,df in df.groupby(['lat','lon']):
    df_list.append(df)
  return df_list

In [ ]:
# create a function to match variable with previous month
### This will be a Figure for the paper
def match_months(df,var_name='VHI',months_prev=1):
  # match each month with previus month(s) to find correlation between them
  # first need to split into list of dfs based on grid cell location
  # also need to create an if statement to only do if -1 is above February 2003
  # append nan for this case
  # and need to create an if statement for Jan 2004-2021 which gets December the previous year instead
  # dont need to do this as already accounted for in df ordering
  df_list = groupby_gc(df)
  #print(df_list[0])
  dfi_list=[]
  for i in range(len(df_list)):
    dfi = df_list[i]
    dfi.reset_index(inplace=True)
    dims = dfi.shape
    #print(dfi.shape)
    pm_var_list=[]
    #print(dfi)
    for j in np.arange(dims[0]):
      if var_name == 'VHI':
        if j >= months_prev:
          pm_var_list.append(dfi.iloc[j-months_prev,3])
        else:
           pm_var_list.append(np.nan)
      elif var_name == 'precip':
        if j >= months_prev:
          pm_var_list.append(dfi.iloc[j-months_prev,6])
        else:
           pm_var_list.append(np.nan)
      elif var_name == 'rtzsm_inst':
        if j >= months_prev:
          pm_var_list.append(dfi.iloc[j-months_prev,7])
        else:
           pm_var_list.append(np.nan)
      elif var_name == 'spei1m':
        if j >= months_prev:
          pm_var_list.append(dfi.iloc[j-months_prev,8])
        else:
           pm_var_list.append(np.nan)
      elif var_name == 'spei2m':
        if j >= months_prev:
          pm_var_list.append(dfi.iloc[j-months_prev,9])
        else:
           pm_var_list.append(np.nan)
      elif var_name == 'spei3m':
        if j >= months_prev:
          pm_var_list.append(dfi.iloc[j-months_prev,10])
        else:
           pm_var_list.append(np.nan)
      elif var_name == 'msdwlwrf':
        if j >= months_prev:
          pm_var_list.append(dfi.iloc[j-months_prev,11])
        else:
          pm_var_list.append(np.nan)
      elif var_name == 't2m':
        if j >= months_prev:
          pm_var_list.append(dfi.iloc[j-months_prev,12])
        else:
          pm_var_list.append(np.nan)
      elif var_name == 'pev':
        if j >= months_prev:
          pm_var_list.append(dfi.iloc[j-months_prev,13])
        else:
          pm_var_list.append(np.nan)
      elif var_name == 'spi_01':
        if j >= months_prev:
          pm_var_list.append(dfi.iloc[j-months_prev,14])
        else:
          pm_var_list.append(np.nan)
      elif var_name == 'spi_02':
        if j >= months_prev:
          pm_var_list.append(dfi.iloc[j-months_prev,15])
        else:
          pm_var_list.append(np.nan)
      elif var_name == 'spi_03':
        if j >= months_prev:
          pm_var_list.append(dfi.iloc[j-months_prev,16])
        else:
           pm_var_list.append(np.nan)
    #print(pm_var_list)
    print(len(pm_var_list))
    dfi[var_name+str(-months_prev)] = pm_var_list
    #print(dfi)
    dfi_list.append(dfi)
  return pd.concat(dfi_list)

In [ ]:
def load_p_file(fname):
  with open(fname,'rb') as input_file:
    e = pickle.load(input_file)
  return e

In [ ]:
# load harvest area filtered by state
mai_ar_state = load_p_file('/content/drive/MyDrive/area_data/crop_grids/mai_harvest_area_by_state.pickle')
soy_ar_state = load_p_file('/content/drive/MyDrive/area_data/crop_grids/soy_harvest_area_by_state.pickle')

In [ ]:
# Need to rename x and y to lat and lon
soy_ar_state = soy_ar_state.rename(columns={'x':'lon','y':'lat'})
mai_ar_state = mai_ar_state.rename(columns={'x':'lon','y':'lat'})

In [ ]:
soy_ar_state = soy_ar_state[soy_ar_state['harvarea']>0]
soy_threshold = np.percentile(soy_ar_state['harvarea'],75)
soy_area_high_ha = soy_ar_state[soy_ar_state['harvarea'] >= soy_threshold]
###
mai_ar_state = mai_ar_state[mai_ar_state['harvarea']>0]
mai_threshold = np.percentile(mai_ar_state['harvarea'],75)
mai_area_high_ha = mai_ar_state[mai_ar_state['harvarea'] >= mai_threshold]
###
high_ha_soy_coords = soy_area_high_ha.drop(columns=['geometry','croparea','qual','set','index_right','UF'])
high_ha_mai_coords = mai_area_high_ha.drop(columns=['geometry','croparea','qual','set','index_right','UF'])

In [ ]:
print(high_ha_soy_coords)

In [ ]:
def match_harvest_areas(df, coords_df):
  # need to put data into per year list before matching
  # harvarea dataframe
  df_list = groupby_gc(df)
  # use df.merge to apply coordinates to each df
  merged_df_list=[]
  for i in range(len(df_list)):
    merged_df_list.append(df_list[i].merge(coords_df,on=['lat','lon']))
  merged_df = pd.concat(merged_df_list)
  return merged_df

In [ ]:
df_soy = match_harvest_areas(df,high_ha_soy_coords)
df_mai = match_harvest_areas(df,high_ha_mai_coords)

In [ ]:
print(df_mai)

In [ ]:
# combine 75% soy and maize - then later cluster based on ccoef relationship with vhi
df_crops = pd.concat([df_mai,df_soy])

In [ ]:
def per_gc_metrics(df):
  m1_df_anom_gc_list = groupby_gc(df)
  lat_list=[]
  lon_list=[]
  vhi_averages=[]
  vhi_spei3_ccoefs=[]
  vhi_rzsm_ccoefs=[]
  vhi_precip_ccoefs=[]
  for i in range(len(m1_df_anom_gc_list)):
    # get average vhi and plot
    vhi_averages.append(np.mean(m1_df_anom_gc_list[i]['VHI']))
    vhi_spei3_ccoefs.append(pearsonr(m1_df_anom_gc_list[i]['VHI'],m1_df_anom_gc_list[i]['spei3m'])[0])
    vhi_rzsm_ccoefs.append(pearsonr(m1_df_anom_gc_list[i]['VHI'],m1_df_anom_gc_list[i]['rtzsm_inst'])[0])
    vhi_precip_ccoefs.append(pearsonr(m1_df_anom_gc_list[i]['VHI'],m1_df_anom_gc_list[i]['precip'])[0])
    # get latitude and longitude coordinates
    lat = m1_df_anom_gc_list[i]['lat'].values[0]
    lat_list.append(lat)
    lon = m1_df_anom_gc_list[i]['lon'].values[0]
    lon_list.append(lon)
  return lat_list, lon_list, vhi_averages, vhi_spei3_ccoefs, vhi_rzsm_ccoefs,vhi_precip_ccoefs

In [ ]:
maize_lats, maize_lons, maize_vhi, maize_speiccoef, maize_rzsm_ccoef, vhi_precip_ccoef = per_gc_metrics(df_crops)

In [ ]:
# add shape feature
# load Brazil shapefile
fname = '/content/drive/MyDrive/shapefiles/BR/BR_EST_SAD69.shp'
shape_feature = ShapelyFeature(Reader(fname).geometries(),
                                ccrs.PlateCarree(), facecolor='none')

In [ ]:
fig = plt.figure(figsize=(10,12))
ax1 = fig.add_subplot(1,1,1,projection=ccrs.PlateCarree())
#ax1.coastlines()
im=ax1.scatter(maize_lons, maize_lats, c=vhi_precip_ccoef, marker=',', transform=ccrs.PlateCarree(), cmap='Spectral',s=4)
# # Set the location and size of the colorbar axes manually
#cax = fig.add_axes([]) # left, bottom, width, height
# # Add the colorbar
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=0.05,axes_class=plt.Axes)
cbar = fig.colorbar(im, ax=ax1,cax=cax)
cbar.ax.set_ylabel('correlation coefficient',fontsize='14')
ax1.title.set_text('correlation between rainfall and vhi')
ax1.set_extent([-80, -30, -40, 10], crs=ccrs.PlateCarree())
#plt.savefig('plots/Iizumi_2020_maize_global.pdf')
#ax1.add_feature(cf.BORDERS)
ax1.add_feature(shape_feature)
ax1.add_feature(cf.LAND)
ax1.add_feature(cf.OCEAN)
ax1.axis('off')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,12))
ax1 = fig.add_subplot(1,1,1,projection=ccrs.PlateCarree())
#ax1.coastlines()
im=ax1.scatter(maize_lons, maize_lats, c=maize_speiccoef, marker=',', transform=ccrs.PlateCarree(), cmap='Spectral',s=4)
# # Set the location and size of the colorbar axes manually
#cax = fig.add_axes([]) # left, bottom, width, height
# # Add the colorbar
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=0.05,axes_class=plt.Axes)
cbar = fig.colorbar(im, ax=ax1,cax=cax)
cbar.ax.set_ylabel('correlation coefficient',fontsize='14')
ax1.title.set_text('correlation between spei and vhi')
ax1.set_extent([-80, -30, -40, 10], crs=ccrs.PlateCarree())
#plt.savefig('plots/Iizumi_2020_maize_global.pdf')
#ax1.add_feature(cf.BORDERS)
ax1.add_feature(shape_feature)
ax1.add_feature(cf.LAND)
ax1.add_feature(cf.OCEAN)
ax1.axis('off')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,12))
ax1 = fig.add_subplot(1,1,1,projection=ccrs.PlateCarree())
#ax1.coastlines()
im=ax1.scatter(maize_lons, maize_lats, c=maize_rzsm_ccoef, marker=',', transform=ccrs.PlateCarree(), cmap='Spectral')
# # Set the location and size of the colorbar axes manually
#cax = fig.add_axes([]) # left, bottom, width, height
# # Add the colorbar
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=0.05,axes_class=plt.Axes)
cbar = fig.colorbar(im, ax=ax1,cax=cax)
cbar.ax.set_ylabel('correlation coefficient',fontsize='14')
ax1.title.set_text('correlation between rzsm and vhi')
ax1.set_extent([-80, -30, -40, 10], crs=ccrs.PlateCarree())
#plt.savefig('plots/Iizumi_2020_maize_global.pdf')
#ax1.add_feature(cf.BORDERS)
ax1.add_feature(shape_feature)
ax1.add_feature(cf.LAND)
ax1.add_feature(cf.OCEAN)
ax1.axis('off')
plt.show()

In [ ]:
def combine_dims(a, i=0, n=1):
  """
  Combines dimensions of numpy array 'a',
  starting at index 'i',
  and combining 'n' dimensions
  """
  s = list(a.shape)
  combined = functools.reduce(lambda x,y: x*y, s[i:i+n+1])
  return np.reshape(a, s[:i] + [combined] + s[i+n+1:])

In [ ]:
# Need to filter df by high harvest area coords


In [ ]:
vhi_df = match_months(df_crops,var_name='VHI')

In [ ]:
print(df_crops)

In [ ]:
df_crops.to_csv('/content/drive/MyDrive/rs_data/merged_dataframe.csv',index=False)

In [ ]:
rzsm_crops_df = match_months(df_crops,var_name='rtzsm_inst')

In [ ]:
print(vhi_df)

In [ ]:
# drop nan values
vhi_df.dropna(inplace=True)
# predict VHI precip, rtzsm spei etc from previous month using range of methods

In [ ]:
# predict VHI from VHI-1
# show error in predictions as RMSE map
print(vhi_df)

In [ ]:
# Do train / test split by year then predict VHI
GBM_model = GradientBoostingRegressor(n_estimators = 200)

In [ ]:
trainx = vhi_df[vhi_df['year']!=2021]['VHI-1'].values.reshape(-1,1)
trainy = vhi_df[vhi_df['year']!=2021]['VHI'].values.reshape(-1,1)

In [ ]:
testx = vhi_df[vhi_df['year']==2021]['VHI-1'].values.reshape(-1,1)
testy = vhi_df[vhi_df['year']==2021]['VHI'].values.reshape(-1,1)

In [ ]:
test_lat = vhi_df[vhi_df['year']==2021]['lat'].values.reshape(-1,1)
test_lon = vhi_df[vhi_df['year']==2021]['lon'].values.reshape(-1,1)

In [ ]:
# next fit model using training data
GBM_model.fit(trainx,trainy)

In [ ]:
GBM_predictions = GBM_model.predict(testx)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
print(mean_absolute_error(testy,GBM_predictions))

In [ ]:
line = np.arange(0,100,10)
fig = plt.figure()
plt.scatter(testy,GBM_predictions,color='green',alpha=0.4,s=6)
plt.ylabel('predicted VHI %')
plt.xlabel('observed VHI %')
plt.title('GBM predictions')
plt.plot(line,line,linestyle='--',color='k')
plt.show()


In [ ]:
#print(pearsonr(testy,GBM_predictions))

In [ ]:
print(test_lat.shape)

In [ ]:
# plot predictions as map (average VHI for 2021)
map_df=pd.DataFrame({'lat':combine_dims(test_lat),'lon':combine_dims(test_lon),'predictions':combine_dims(GBM_predictions),'observed':combine_dims(testy)})

In [ ]:
def rmse(predictions, targets):
    return ((np.sqrt(((predictions - targets) ** 2)).mean())/np.mean(targets))*100

In [ ]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [ ]:
def get_locationwise_av(df):
  df_list = groupby_gc(df)
  av_vhi_preds=[]
  av_vhi_obs=[]
  vhi_rmse=[]
  lats=[]
  lons=[]
  for i in range(len(df_list)):
    av_vhi_preds.append(np.mean(df_list[i]['predictions']))
    av_vhi_obs.append(np.mean(df_list[i]['observed']))
    vhi_rmse.append(rmse(df_list[i]['predictions'],df_list[i]['observed']))
    lats.append(df_list[i]['lat'].values[0])
    lons.append(df_list[i]['lon'].values[0])
  return vhi_rmse,lats,lons, av_vhi_preds, av_vhi_obs

In [ ]:
vhi_rmse_map,map_lats,map_lons,preds,obs = get_locationwise_av(map_df)

In [ ]:
# load Brazil shapefile
fname = '/content/drive/MyDrive/shapefiles/BR/BR_EST_SAD69.shp'
shape_feature = ShapelyFeature(Reader(fname).geometries(),
                                ccrs.PlateCarree(), facecolor='none')

In [ ]:
fig = plt.figure(figsize=(10,12))
ax1 = fig.add_subplot(1,1,1,projection=ccrs.PlateCarree())
#ax1.coastlines()
im=ax1.scatter(map_lons, map_lats, c=vhi_rmse_map, marker=',', transform=ccrs.PlateCarree(), cmap='Spectral_r',s=4)
# # Set the location and size of the colorbar axes manually
#cax = fig.add_axes([]) # left, bottom, width, height
# # Add the colorbar
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=0.05,axes_class=plt.Axes)
cbar = fig.colorbar(im, ax=ax1,cax=cax)
cbar.ax.set_ylabel('VHI(%) RMSE',fontsize='14')
ax1.title.set_text('RMSE VHI predictions (all months 2021)')
ax1.set_extent([-80, -30, -40, 10], crs=ccrs.PlateCarree())
#plt.savefig('plots/Iizumi_2020_maize_global.pdf')
#ax1.add_feature(cf.BORDERS)
ax1.add_feature(shape_feature)
ax1.add_feature(cf.LAND)
ax1.add_feature(cf.OCEAN)
ax1.axis('off')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,12))
ax1 = fig.add_subplot(1,1,1,projection=ccrs.PlateCarree())
#ax1.coastlines()
im=ax1.scatter(map_lons, map_lats, c=preds, marker=',', transform=ccrs.PlateCarree(), cmap='Spectral',vmin=0,vmax=70)
# # Set the location and size of the colorbar axes manually
#cax = fig.add_axes([]) # left, bottom, width, height
# # Add the colorbar
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=0.05,axes_class=plt.Axes)
cbar = fig.colorbar(im, ax=ax1,cax=cax)
cbar.ax.set_ylabel('Predicted VHI',fontsize='14')
ax1.title.set_text('Average VHI predictions (2021)')
ax1.set_extent([-80, -30, -40, 10], crs=ccrs.PlateCarree())
#plt.savefig('plots/Iizumi_2020_maize_global.pdf')
#ax1.add_feature(cf.BORDERS)
ax1.add_feature(shape_feature)
ax1.add_feature(cf.LAND)
ax1.add_feature(cf.OCEAN)
ax1.axis('off')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,12))
ax1 = fig.add_subplot(1,1,1,projection=ccrs.PlateCarree())
#ax1.coastlines()
im=ax1.scatter(map_lons, map_lats, c=obs, marker=',', transform=ccrs.PlateCarree(), cmap='Spectral',vmin=0,vmax=70)
# # Set the location and size of the colorbar axes manually
#cax = fig.add_axes([]) # left, bottom, width, height
# # Add the colorbar
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=0.05,axes_class=plt.Axes)
cbar = fig.colorbar(im, ax=ax1,cax=cax)
cbar.ax.set_ylabel('Predicted VHI',fontsize='14')
ax1.title.set_text('Average observed VHI (2021)')
ax1.set_extent([-80, -30, -40, 10], crs=ccrs.PlateCarree())
#plt.savefig('plots/Iizumi_2020_maize_global.pdf')
#ax1.add_feature(cf.BORDERS)
ax1.add_feature(shape_feature)
ax1.add_feature(cf.LAND)
ax1.add_feature(cf.OCEAN)
ax1.axis('off')
plt.show()

In [ ]:
print(vhi_df)

In [ ]:
print(df_crops)

In [ ]:
df_crops.to_csv('/content/drive/MyDrive/df_crops_full.csv',index=False)

NameError: name 'df_crops' is not defined

In [ ]:
vhi_df = match_months(df_crops,var_name='VHI')
vhi_df.dropna(inplace=True)
precip_df = match_months(df_crops,var_name='precip')
precip_df.dropna(inplace=True)
rzsm_df = match_months(df_crops,var_name='rtzsm_inst')
rzsm_df.dropna(inplace=True)
spei1_df = match_months(df_crops,var_name='spei1m')
spei1_df.dropna(inplace=True)
spei2_df = match_months(df_crops,var_name='spei2m')
spei2_df.dropna(inplace=True)
spei3_df = match_months(df_crops,var_name='spei3m')
spei3_df.dropna(inplace=True)

In [ ]:
print(vhi_df)

In [ ]:
spi1_df = match_months(df_crops,var_name='spi_01')
spi1_df.dropna(inplace=True)
spi2_df = match_months(df_crops,var_name='spi_02')
spi2_df.dropna(inplace=True)
spi3_df = match_months(df_crops,var_name='spi_03')
spi3_df.dropna(inplace=True)

In [ ]:
# Get era5 data
longrad_df = match_months(df_crops,var_name='msdwlwrf')
longrad_df.dropna(inplace=True)
pev_df = match_months(df_crops,var_name='pev')
pev_df.dropna(inplace=True)
t2m_df = match_months(df_crops,var_name='t2m')
t2m_df.dropna(inplace=True)

In [ ]:
print(vhi_df['lon'].values.shape)

In [ ]:
print(spi1_df)

In [ ]:
# create prediction dataframe out of -1s and coordinates, year and month
m1_df = pd.DataFrame({'lon':vhi_df['lon'].values,'lat':vhi_df['lat'].values,'vhi-1':vhi_df['VHI-1'].values,'precip-1':precip_df['precip-1'].values,'rzsm-1':rzsm_df['rtzsm_inst-1'].values,
                      'spei1-1m':spei1_df['spei1m-1'].values,'spei2-1m':spei2_df['spei2m-1'].values,'spei3-1m':spei3_df['spei3m-1'].values,'spi01-1':spi1_df['spi_01-1'].values,'spi02-1':spi2_df['spi_02-1'].values,
                      'spi03-1':spi3_df['spi_03-1'].values,'pev-1':pev_df['pev-1'].values,'longrad-1':longrad_df['msdwlwrf-1'].values,'t2m-1':t2m_df['t2m-1'].values,
                      'month':vhi_df['month'].values,'year':vhi_df['year'].values,'VHI':vhi_df['VHI'].values})

In [ ]:
print(m1_df)

In [ ]:
#save m1_df dataframe
m1_df.to_csv('/content/drive/MyDrive/VHI_spei_rzsm_dataset.csv',index=False)

In [ ]:
# create xy train-test split
# added meteorological data to model
# 2021 is test year
m1_df_test = m1_df[m1_df['year']==2021]
m1_df_train = m1_df[m1_df['year']<2021]

##
trainx = m1_df_train.drop(columns=['VHI'])
testx = m1_df_test.drop(columns=['VHI'])
# trainy and testy
trainy = m1_df_train['VHI']
testy =  m1_df_test['VHI']

In [ ]:
# Do train / test split by year then predict VHI
GBM_model = GradientBoostingRegressor(n_estimators = 200)
# fit GBM model to train and test
# next fit model using training data
GBM_model.fit(trainx,trainy) # model with vhi and spei precip and rzsm from previous month

In [ ]:
m_GBM_predictions = GBM_model.predict(testx)

In [ ]:
line = np.arange(0,100,10)
fig = plt.figure()
plt.scatter(testy,m_GBM_predictions,color='green',alpha=0.4,s=6)
plt.ylabel('predicted VHI %')
plt.xlabel('observed VHI %')
plt.plot(line,line,linestyle='--',color='k')
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
print(mean_absolute_error(testy,m_GBM_predictions))

In [ ]:
print(pearsonr(testy,m_GBM_predictions))

In [ ]:
from sklearn.inspection import permutation_importance
result_0 = permutation_importance(GBM_model,trainx,trainy,scoring='r2')

In [ ]:
gbm_importances = pd.Series(result_0.importances_mean, index=trainx.columns)
fig, ax = plt.subplots()
gbm_importances.plot.bar(yerr=result_0.importances_std, ax=ax)
ax.set_title("GBM feature importances all data")
ax.set_ylabel("r2 score")
fig.tight_layout()
plt.ylim(0,1)
plt.show()

In [ ]:
# Do permutation feature importance:


In [ ]:
# remove variables and check
# 2021 is test year
m1_df_test = m1_df[m1_df['year']==2021]
m1_df_train = m1_df[m1_df['year']<2021]

##
trainx = m1_df_train.drop(columns=['VHI','vhi-1'])
testx = m1_df_test.drop(columns=['VHI','vhi-1'])
# trainy and testy
trainy = m1_df_train['VHI']
testy =  m1_df_test['VHI']

In [ ]:
# Do train / test split by year then predict VHI
GBM_model = GradientBoostingRegressor(n_estimators = 200)
# fit GBM model to train and test
# next fit model using training data
GBM_model.fit(trainx,trainy)

In [ ]:
m_GBM_predictions = GBM_model.predict(testx)

In [ ]:
line = np.arange(0,100,10)
fig = plt.figure()
plt.scatter(testy,m_GBM_predictions,color='green',alpha=0.4,s=6)
plt.ylabel('predicted VHI')
plt.xlabel('observed VHI')
plt.plot(line,line,linestyle='--',color='k')
plt.show()
print(pearsonr(testy,m_GBM_predictions))

In [ ]:
# Use GBM model to predict future months of SPEI3 and/or rzsm then add make model based on that
m1_df = pd.DataFrame({'lon':vhi_df['lon'].values,'lat':vhi_df['lat'].values,'vhi-1':vhi_df['VHI-1'].values,'precip-1':precip_df['precip-1'].values,'rzsm-1':rzsm_df['rtzsm_inst-1'].values,'spei1-1m':spei1_df['spei1m-1'].values,'spei3-1m':spei3_df['spei3m-1'].values,'month':vhi_df['month'].values,'year':vhi_df['year'].values,'VHI':vhi_df['VHI'].values,'rzsm':rzsm_df['rtzsm_inst'].values})
m1_df_test = m1_df[m1_df['year']==2021]
m1_df_train = m1_df[m1_df['year']<2021]

##
trainx = m1_df_train.drop(columns=['VHI','vhi-1','rzsm'])
testx = m1_df_test.drop(columns=['VHI','vhi-1','rzsm'])
# trainy and testy
trainy = m1_df_train['rzsm']
testy =  m1_df_test['rzsm']

In [ ]:
# Do train / test split by year then predict VHI
GBM_model = GradientBoostingRegressor(n_estimators = 200)
# fit GBM model to train and test
# next fit model using training data
GBM_model.fit(trainx,trainy)

In [ ]:
m_GBM_predictions = GBM_model.predict(testx)

In [ ]:
line = np.arange(0,100,10)
fig = plt.figure()
plt.scatter(testy,m_GBM_predictions,color='green',alpha=0.4,s=6)
plt.ylabel('predicted RZSM')
plt.xlabel('observed RZSM')
plt.plot(line,line,linestyle='--',color='k')
plt.show()
print(pearsonr(testy,m_GBM_predictions))

Unpredicting RZSM is probably due to the downward trend in RZSM across the time period. to improve this should detrend RZSM

In [ ]:
# detrend RZSM
print(m1_df)

In [ ]:
# Is correlation between predicted RZSM and next months VHI better than VHI and previous months RZSM
#line = np.arange(0,100,10)
fig = plt.figure()
plt.scatter(m1_df_test['VHI'],m_GBM_predictions,color='green',alpha=0.4,s=6)
plt.ylabel('predicted RZSM')
plt.xlabel('observed VHI')
#plt.plot(line,line,linestyle='--',color='k')
plt.show()
print(pearsonr(m1_df_test['VHI'],m_GBM_predictions))

In [ ]:
# how does it compare to previous months rzsm
fig = plt.figure()
plt.scatter(m1_df_test['VHI'],m1_df_test['rzsm-1'].values,color='green',alpha=0.4,s=6)
plt.ylabel('RZSM-1')
plt.xlabel('observed VHI')
#plt.plot(line,line,linestyle='--',color='k')
plt.show()
print(pearsonr(m1_df_test['VHI'],m1_df_test['rzsm-1'].values))

In [ ]:
# how does it compare to same months rzsm
fig = plt.figure()
plt.scatter(m1_df_test['VHI'],m1_df_test['rzsm'].values,color='green',alpha=0.4,s=6)
plt.ylabel('observed RZSM')
plt.xlabel('observed VHI')
#plt.plot(line,line,linestyle='--',color='k')
plt.show()
print(pearsonr(m1_df_test['VHI'],m1_df_test['rzsm'].values))

In [ ]:
# how does it compare to same months rzsm
fig = plt.figure()
plt.scatter(m1_df_test['vhi-1'],m1_df_test['spei3-1m'].values,color='green',alpha=0.4,s=6)
plt.ylabel('observed spei3m')
plt.xlabel('observed VHI')
#plt.plot(line,line,linestyle='--',color='k')
plt.show()
print(pearsonr(m1_df_test['vhi-1'],m1_df_test['spei3-1m'].values))

Need to detrend and take anomalies for better results from meteorological variables and soil moisture correlation with VHI. Also need to detrend and take VHI as anomaly

In [ ]:
# detrend (if needed and get vars as spatial anomaly)
print(m1_df)

In [ ]:
def get_anomaly(df,var='VHI'):
  dfgc = groupby_gc(df)
  # groupby grid cell
  for i in range(len(dfgc)):
    gc_mean = np.mean(dfgc[i][var])
    gc_anom = dfgc[i][var] - gc_mean
    dfgc[i][var+' anom'] = gc_anom
  return pd.concat(dfgc)

In [ ]:
m1_df = pd.DataFrame({'lon':vhi_df['lon'].values,'lat':vhi_df['lat'].values,'vhi-1':vhi_df['VHI-1'].values,'precip-1':precip_df['precip-1'].values,'rzsm-1':rzsm_df['rtzsm_inst-1'].values,'spei1-1m':spei1_df['spei1m-1'].values,'spei3-1m':spei3_df['spei3m-1'].values,'spei3':spei3_df['spei3m'].values,'month':vhi_df['month'].values,'year':vhi_df['year'].values,'VHI':vhi_df['VHI'].values,'rzsm':rzsm_df['rtzsm_inst'].values})

m1_df_anom = get_anomaly(m1_df)
m1_df_anom = get_anomaly(m1_df_anom,'rzsm-1')
m1_df_anom = get_anomaly(m1_df_anom,'precip-1')
m1_df_anom = get_anomaly(m1_df_anom,'vhi-1')
print(m1_df_anom)

In [ ]:

fig = plt.figure()
plt.scatter(m1_df_anom['VHI anom'].values,m1_df_anom['spei3'].values,color='green',alpha=0.4,s=6)
plt.ylabel('observed spei3m')
plt.xlabel('observed VHI')
#plt.plot(line,line,linestyle='--',color='k')
plt.show()
print(pearsonr(m1_df_anom['VHI anom'],m1_df_anom['spei3'].values))

In [ ]:
m1_df_test = m1_df_anom[m1_df_anom['year']==2021]
m1_df_train = m1_df_anom[m1_df_anom['year']<2021]

##
trainx = m1_df_train.drop(columns=['VHI','vhi-1','rzsm','precip-1','rzsm-1','VHI anom','spei3'])
testx = m1_df_test.drop(columns=['VHI','vhi-1','rzsm','precip-1','rzsm-1','VHI anom','spei3'])
# trainy and testy
trainy = m1_df_train['VHI anom']
testy =  m1_df_test['VHI anom']
print(testx)

In [ ]:
# Do train / test split by year then predict VHI
GBM_model = GradientBoostingRegressor(n_estimators = 200)
# fit GBM model to train and test
# next fit model using training data
GBM_model.fit(trainx,trainy)

In [ ]:
m_GBM_predictions = GBM_model.predict(testx)

In [ ]:
line = np.arange(-40,40,1)
fig = plt.figure()
plt.scatter(testy,m_GBM_predictions,color='green',alpha=0.4,s=6)
plt.ylabel('predicted VHI anomaly')
plt.xlabel('observed VHI anomaly')
plt.plot(line,line,linestyle='--',color='k')
plt.show()
print(pearsonr(testy,m_GBM_predictions))

In [ ]:
m1_df_test = m1_df_anom[m1_df_anom['year']==2021]
m1_df_train = m1_df_anom[m1_df_anom['year']<2021]

##
trainx = m1_df_train.drop(columns=['VHI','vhi-1','rzsm','precip-1','rzsm-1','VHI anom','spei3','vhi-1 anom'])
testx = m1_df_test.drop(columns=['VHI','vhi-1','rzsm','precip-1','rzsm-1','VHI anom','spei3','vhi-1 anom'])
# trainy and testy
trainy = m1_df_train['VHI anom']
testy =  m1_df_test['VHI anom']
print(testx)

In [ ]:
# Do train / test split by year then predict VHI
GBM_model = GradientBoostingRegressor(n_estimators = 200)
# fit GBM model to train and test
# next fit model using training data
GBM_model.fit(trainx,trainy)

In [ ]:
m_GBM_predictions = GBM_model.predict(testx)

In [ ]:
line = np.arange(-40,40,1)
fig = plt.figure()
plt.scatter(testy,m_GBM_predictions,color='green',alpha=0.4,s=6)
plt.ylabel('predicted VHI anomaly')
plt.xlabel('observed VHI anomaly')
plt.plot(line,line,linestyle='--',color='k')
plt.show()
print(pearsonr(testy,m_GBM_predictions))

In [ ]:
# now plot observed in same year
m1_df_test = m1_df_anom[m1_df_anom['year']==2021]
m1_df_train = m1_df_anom[m1_df_anom['year']<2021]
#print(m1_df_test)
##
trainx = m1_df_train.drop(columns=['VHI','vhi-1','precip-1','rzsm-1','VHI anom','vhi-1 anom','spei3-1m','spei1-1m','rzsm-1 anom','precip-1 anom'])
testx = m1_df_test.drop(columns=['VHI','vhi-1','precip-1','rzsm-1','VHI anom','vhi-1 anom','spei3-1m','spei1-1m','rzsm-1 anom','precip-1 anom'])
# trainy and testy
trainy = m1_df_train['VHI anom']
testy =  m1_df_test['VHI anom']
print(trainx)
print(testx)

In [ ]:
# Do train / test split by year then predict VHI
GBM_model = GradientBoostingRegressor(n_estimators = 200)
# fit GBM model to train and test
# next fit model using training data
GBM_model.fit(trainx,trainy)

In [ ]:
m_GBM_predictions = GBM_model.predict(testx)

In [ ]:
line = np.arange(-40,40,1)
fig = plt.figure()
plt.scatter(testy,m_GBM_predictions,color='green',alpha=0.4,s=6)
plt.ylabel('predicted VHI anomaly')
plt.xlabel('observed VHI anomaly')
plt.plot(line,line,linestyle='--',color='k')
plt.show()
print(pearsonr(testy,m_GBM_predictions))

In [ ]:
print(m1_df_anom)

In [ ]:
# get average rzsm, vhi, spei etc per grid cell
m1_df_anom_gc_list = groupby_gc(m1_df_anom)

In [ ]:
# fuction for correlations and averages for each grid cell
def per_gc_metrics(df):
  m1_df_anom_gc_list = groupby_gc(df)
  lat_list=[]
  lon_list=[]
  vhi_averages=[]
  vhi_spei3_ccoefs=[]
  vhi_rzsm_ccoefs=[]
  for i in range(len(m1_df_anom_gc_list)):
    # get average vhi and plot
    vhi_averages.append(np.mean(m1_df_anom_gc_list[i]['VHI']))
    vhi_spei3_ccoefs.append(pearsonr(m1_df_anom_gc_list[i]['VHI'],m1_df_anom_gc_list[i]['spei3'])[0])
    vhi_rzsm_ccoefs.append(pearsonr(m1_df_anom_gc_list[i]['VHI'],m1_df_anom_gc_list[i]['rzsm'])[0])
    # get latitude and longitude coordinates
    lat = m1_df_anom_gc_list[i]['lat'].values[0]
    lat_list.append(lat)
    lon = m1_df_anom_gc_list[i]['lon'].values[0]
    lon_list.append(lon)
  return lat_list, lon_list, vhi_averages, vhi_spei3_ccoefs, vhi_rzsm_ccoefs

In [ ]:
lat_list, lon_list, vhi_avs, vhi_spei3_ccoefs, vhi_rzsm_ccoefs = per_gc_metrics(m1_df_anom)

In [ ]:
fig = plt.figure(figsize=(10,12))
ax1 = fig.add_subplot(1,1,1,projection=ccrs.PlateCarree())
#ax1.coastlines()
im=ax1.scatter(lon_list, lat_list, c=vhi_avs, marker=',', transform=ccrs.PlateCarree(), cmap='Spectral',vmin=0,vmax=70)
# # Set the location and size of the colorbar axes manually
#cax = fig.add_axes([]) # left, bottom, width, height
# # Add the colorbar
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=0.05,axes_class=plt.Axes)
cbar = fig.colorbar(im, ax=ax1,cax=cax)
cbar.ax.set_ylabel('average VHI',fontsize='14')
ax1.title.set_text('Average VHI (2021)')
ax1.set_extent([-80, -30, -40, 10], crs=ccrs.PlateCarree())
#plt.savefig('plots/Iizumi_2020_maize_global.pdf')
#ax1.add_feature(cf.BORDERS)
ax1.add_feature(shape_feature)
ax1.add_feature(cf.LAND)
ax1.add_feature(cf.OCEAN)
ax1.axis('off')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,12))
ax1 = fig.add_subplot(1,1,1,projection=ccrs.PlateCarree())
#ax1.coastlines()
im=ax1.scatter(lon_list, lat_list, c=vhi_spei3_ccoefs, marker=',', transform=ccrs.PlateCarree(), cmap='Spectral')
# # Set the location and size of the colorbar axes manually
#cax = fig.add_axes([]) # left, bottom, width, height
# # Add the colorbar
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=0.05,axes_class=plt.Axes)
cbar = fig.colorbar(im, ax=ax1,cax=cax)
cbar.ax.set_ylabel('correlation coefficient',fontsize='14')
ax1.title.set_text('correlation between spei3 and vhi')
ax1.set_extent([-80, -30, -40, 10], crs=ccrs.PlateCarree())
#plt.savefig('plots/Iizumi_2020_maize_global.pdf')
#ax1.add_feature(cf.BORDERS)
ax1.add_feature(shape_feature)
ax1.add_feature(cf.LAND)
ax1.add_feature(cf.OCEAN)
ax1.axis('off')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,12))
ax1 = fig.add_subplot(1,1,1,projection=ccrs.PlateCarree())
#ax1.coastlines()
im=ax1.scatter(lon_list, lat_list, c=vhi_rzsm_ccoefs, marker=',', transform=ccrs.PlateCarree(), cmap='Spectral')
# # Set the location and size of the colorbar axes manually
#cax = fig.add_axes([]) # left, bottom, width, height
# # Add the colorbar
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=0.05,axes_class=plt.Axes)
cbar = fig.colorbar(im, ax=ax1,cax=cax)
cbar.ax.set_ylabel('correlation coefficient',fontsize='14')
ax1.title.set_text('correlation between rzsm and vhi')
ax1.set_extent([-80, -30, -40, 10], crs=ccrs.PlateCarree())
#plt.savefig('plots/Iizumi_2020_maize_global.pdf')
#ax1.add_feature(cf.BORDERS)
ax1.add_feature(shape_feature)
ax1.add_feature(cf.LAND)
ax1.add_feature(cf.OCEAN)
ax1.axis('off')
plt.show()

In [ ]:

print(m1_df_anom_gc_list[0])

In [ ]:
#### Forward months

# create a function to match variable with previous month
### This will be a Figure for the paper
def match_month_forward(df,var_name='VHI',months_plus=1):
  # match each month with previus month(s) to find correlation between them
  # first need to split into list of dfs based on grid cell location
  # also need to create an if statement to only do if -1 is above February 2003
  # append nan for this case
  # and need to create an if statement for Jan 2004-2021 which gets December the previous year instead
  # dont need to do this as already accounted for in df ordering
  # Changed function to give future rather than past months.
  # Need to check if this works properly

  # Need to use if statement to cut off last part

  df_list = groupby_gc(df)
  #print(df_list[0])
  dfi_list=[]
  for i in range(len(df_list)):
    dfi = df_list[i]
    dfi.reset_index(inplace=True)
    dims = dfi.shape
    #print(dfi.shape)
    pm_var_list=[]
    #print(dfi)
    for j in np.arange(dims[0]):
      if var_name == 'VHI':
        if j >= months_plus:
          pm_var_list.append(dfi.iloc[j+months_plus,3])
        else:
           pm_var_list.append(np.nan)
      elif var_name == 'precip':
        if j >= months_plus:
          pm_var_list.append(dfi.iloc[j+months_plus,6])
        else:
           pm_var_list.append(np.nan)
      elif var_name == 'rtzsm_inst':
        if j >= months_plus:
          pm_var_list.append(dfi.iloc[j+months_plus,7])
        else:
           pm_var_list.append(np.nan)
      elif var_name == 'spei1m':
        if j >= months_plus:
          pm_var_list.append(dfi.iloc[j+months_plus,8])
        else:
           pm_var_list.append(np.nan)
      elif var_name == 'spei3m':
        if j >= months_plus:
          pm_var_list.append(dfi.iloc[j+months_plus,9])
        else:
           pm_var_list.append(np.nan)
    #print(pm_var_list)
    dfi[var_name+str(+months_plus)] = pm_var_list
    #print(dfi)
    dfi_list.append(dfi)
  return pd.concat(dfi_list)

In [ ]:
#
#df_soy_nm = match_month_forward(df_soy)


In [ ]:
# fit selection of simple ML models, show model evaluation broken down by region and (if time) cluster of correlation

In [ ]:
df_crops2=pd.concat(m1_df_anom_gc_list)

In [ ]:
print(df_crops2)

In [ ]:
# Get train and test and select features
features_df_vhi = pd.DataFrame({'lat':df_crops2['lat'],'lon':df_crops2['lon'],'rzsm':df_crops2['rzsm-1'],'spei3':df_crops2['spei3-1m'],'spei1':df_crops2['spei1-1m'],'precip':df_crops2['precip-1 anom'],'month':df_crops2['month'],'year':df_crops2['year'],
                                'vhi':df_crops2['VHI']})

features_df_vhi_anom = pd.DataFrame({'lat':df_crops2['lat'],'lon':df_crops2['lon'],'rzsm':df_crops2['rzsm-1'],'spei3':df_crops2['spei3-1m'],'spei1':df_crops2['spei1-1m'],'precip':df_crops2['precip-1 anom'],'month':df_crops2['month'],'year':df_crops2['year'],
                                'vhi':df_crops2['VHI anom']})

In [ ]:
#create train/test splits
# select 3 random years
yrs_to_sample = np.arange(2003,2021+1)
print(yrs_to_sample)
sample_yrs =np.random.choice(yrs_to_sample,5)
print(sample_yrs)

In [ ]:
# Get 5 sets of training and testing sets
def train_test_splits(features_df,sample_yrs):
  train_set_list=[]
  test_set_list=[]
  for i in range(len(sample_yrs)):
    train_seti = features_df[features_df['year']!=sample_yrs[i]]
    test_seti = features_df[features_df['year']==sample_yrs[i]]
    train_set_list.append(train_seti)
    test_set_list.append(test_seti)
  return train_set_list,test_set_list

In [ ]:
train_sets, test_sets = train_test_splits(features_df_vhi,sample_yrs)

In [ ]:
def fit_gbm_splits(trains, tests,target_var):
  predictions_list=[]
  for i in range(len(trains)):
    trainxi = trains[i].drop(columns=[target_var])
    testxi =  tests[i].drop(columns=[target_var])
    # Get y data
    trainyi = trains[i][target_var]
    testyi = tests[i][target_var]
    #
    GBM_model = GradientBoostingRegressor(n_estimators=200)
    GBM_model.fit(trainxi,trainyi)
    print('model: ',i,' trained')
    ## Return set of predictions for each by GBM model
    predictionsi = GBM_model.predict(testxi)
    predictions_list.append(predictionsi)
  return predictions_list


In [ ]:
#gbm_predictions = fit_gbm_splits(train_sets,test_sets,target_var='vhi')

Train model and then evaluate based on region

In [ ]:
# save GBM model predictions
def make_output(test_sets,test_yrs,gbm_predictions):
  gbm_output=[]
  for i in range(len(gbm_predictions)):
    gbmpi = gbm_predictions[i]
    testyr_list = [test_yrs[i]]*len(gbmpi)
    test_sets[i]['test year'] = testyr_list
    test_sets[i]['predictions'] = gbmpi
    test_sets[i]['model run'] = [i]*len(gbmpi)
    gbm_output.append(test_sets[i])

  return pd.concat(gbm_output)

In [ ]:
#gbmOutput = make_output(test_sets,sample_yrs,gbm_predictions)

In [ ]:
#print(gbmOutput)

In [ ]:
# save dataframe
#gbmOutput.to_csv('/content/drive/MyDrive/GBM_model_output.csv',index=False)

In [ ]:
def fit_rfr_splits(trains, tests,target_var):
  predictions_list=[]
  for i in range(len(trains)):
    trainxi = trains[i].drop(columns=[target_var])
    testxi =  tests[i].drop(columns=[target_var])
    # Get y data
    trainyi = trains[i][target_var]
    testyi = tests[i][target_var]
    #
    GBM_model = RandomForestRegressor(n_estimators=100)
    GBM_model.fit(trainxi,trainyi)
    print('model: ',i,' trained')
    ## Return set of predictions for each by GBM model
    predictionsi = GBM_model.predict(testxi)
    predictions_list.append(predictionsi)
  return predictions_list

In [ ]:
# Need to keep test years the same as initial GBM model run for comparison
sample_yrs = [2019, 2008, 2017, 2003, 2012]
train_sets, test_sets = train_test_splits(features_df_vhi,sample_yrs)

In [ ]:
rfr_predictions = fit_rfr_splits(train_sets,test_sets,target_var='vhi')

In [ ]:
rfrOutput = make_output(test_sets,sample_yrs,rfr_predictions)

In [ ]:
rfrOutput.to_csv('/content/drive/MyDrive/rfr_model_output.csv',index=False)

In [ ]:
# Do linear regression
from sklearn.preprocessing import RobustScaler

def fit_mlr_splits(trains, tests,target_var):
  predictions_list=[]
  for i in range(len(trains)):
    trainxi = trains[i].drop(columns=[target_var])
    testxi =  tests[i].drop(columns=[target_var])
    # Get y data
    trainyi = trains[i][target_var]
    testyi = tests[i][target_var]
    #
    GBM_model = LinearRegression()
    scaler = RobustScaler()
    trainxi = scaler.fit_transform(trainxi)
    GBM_model.fit(trainxi,trainyi)
    print('model: ',i,' trained')
    ## Return set of predictions for each by GBM model
    print(testxi)
    testxi = scaler.fit_transform(testxi)
    print(testxi)
    predictionsi = GBM_model.predict(testxi)
    predictions_list.append(predictionsi)
  return predictions_list

In [ ]:
train_sets, test_sets = train_test_splits(features_df_vhi,sample_yrs)
mlr_predictions = fit_mlr_splits(train_sets,test_sets,target_var='vhi')

In [ ]:
mlrOutput = make_output(test_sets,sample_yrs,mlr_predictions)

In [ ]:
mlrOutput.to_csv('/content/drive/MyDrive/mlr_model_output.csv',index=False)

In [ ]:
from sklearn.svm import SVR

In [ ]:
def fit_svr_splits(trains, tests,target_var):
  predictions_list=[]
  for i in range(len(trains)):
    trainxi = trains[i].drop(columns=[target_var])
    testxi =  tests[i].drop(columns=[target_var])
    # Get y data
    trainyi = trains[i][target_var]
    testyi = tests[i][target_var]
    #
    GBM_model = SVR()
    scaler = RobustScaler()
    trainxi = scaler.fit_transform(trainxi)
    GBM_model.fit(trainxi,trainyi)
    print('model: ',i,' trained')
    ## Return set of predictions for each by GBM model
    testxi = scaler.fit_transform(testxi)
    predictionsi = GBM_model.predict(testxi)
    predictions_list.append(predictionsi)
  return predictions_list

In [ ]:
train_sets, test_sets = train_test_splits(features_df_vhi,sample_yrs)
svr_predictions = fit_svr_splits(train_sets,test_sets,target_var='vhi')

In [ ]:
svrOutput = make_output(test_sets,sample_yrs,svr_predictions)

In [ ]:
svrOutput.to_csv('/content/drive/MyDrive/svr_model_output.csv',index=False)

In [ ]:
# bring in code for dense neural network
import keras
from keras import layers
from keras import ops

In [ ]:
inputs = keras.Input(shape=(784,)) # start neural network with input node with shape containing features
dense = layers.Dense(64, activation="relu")
x = dense(inputs)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(10)(x)
